In [81]:
import json

input_file = "C:/Users/jguo/Desktop/PURE-main/data/new_test.jsonl"
output_file = "C:/Users/jguo/Desktop/PURE-main/data/test_filtered_landmarktype.jsonl"

def fix_landmark_type(triples):
    new_triples = []
    for triple in triples:
        if triple.get("rel") == "isLandmarkType":
            # 取地名类型前缀（如 "Rue"）
            sub = triple.get("sub", "")
            prefix = sub.split()[0] if sub else ""
            # 新 triple
            new_triples.append({
                "sub": prefix,
                "rel": "isLandmarkTypeOf",
                "obj": sub.lower()
            })
        else:
            new_triples.append(triple)
    return new_triples

with open(input_file, "r", encoding="utf-8") as fin, open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        data = json.loads(line)
        if "triples" in data:
            data["triples"] = fix_landmark_type(data["triples"])
        fout.write(json.dumps(data, ensure_ascii=False) + "\n")

print("Traitement terminé !")

Traitement terminé !


In [82]:
import re
import json
from unidecode import unidecode
import dateparser

def extract_full_dates(text):
    """
    Extraire toutes les expressions de dates complètes dans la phrase (retourne une liste de segments d'origine, format français)
    """
    # Correspond à "1er février 1877", "2 mars 1912", "01 avril 1855", etc.
    pattern = r"\b(?:1er|\d{1,2})\s+[a-zéû]+(?:\s+\d{4})"
    results = []
    for m in re.finditer(pattern, text, re.IGNORECASE):
        phrase = text[m.start():m.end()]
        # On ne garde que celles qui peuvent être correctement analysées par dateparser
        dt = dateparser.parse(phrase, languages=["fr"])
        if dt:
            results.append(phrase.strip())
    return results

def replace_incomplete_dates(sent, triples):
    # Extraire toutes les dates complètes dans la phrase (ex : « 1er février 1877 »)
    full_dates = extract_full_dates(unidecode(sent))
    # Construire une table de correspondance { (année, mois): expression d'origine }
    mapping = {}
    for d in full_dates:
        dt = dateparser.parse(d, languages=["fr"])
        if dt:
            key = (str(dt.year), str(dt.month).zfill(2))
            mapping[key] = d.strip()

    # Traiter chaque triple individuellement
    for triple in triples:
        obj = triple.get("obj", "")
        m = re.fullmatch(r"(\d{4})-(\d{2})", obj)
        if m:
            year, month = m.group(1), m.group(2)
            if (year, month) in mapping:
                # Remplacer par l'expression temporelle réelle trouvée dans la phrase
                triple["obj"] = mapping[(year, month)]
        # Pour traiter aussi les formats yyyy-mm-dd si besoin
        m2 = re.fullmatch(r"(\d{4})-(\d{2})-(\d{2})", obj)
        if m2:
            year, month, day = m2.group(1), m2.group(2), int(m2.group(3))
            for dstr, dphrase in mapping.items():
                if dstr == (year, month):
                    if str(day) in dphrase or f"{day:02d}" in dphrase or (day == 1 and "1er" in dphrase):
                        triple["obj"] = dphrase
    return triples

# ====== Traitement batch du fichier ======

input_file = "C:/Users/jguo/Desktop/PURE-main/data/test_filtered_landmarktype.jsonl"    # Chemin du fichier d'entrée
output_file = "C:/Users/jguo/Desktop/PURE-main/data/test_fixed.jsonl"  # Chemin du fichier de sortie

with open(input_file, "r", encoding="utf-8") as fin, open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        d = json.loads(line)
        d["triples"] = replace_incomplete_dates(d["sent"], d["triples"])
        fout.write(json.dumps(d, ensure_ascii=False) + "\n")

print("Traitement terminé !")


Traitement terminé !


In [83]:
import json
import dateparser
import re

def fr_date_to_iso(date_str):
    # Convertit une date en français (« 1er février 1877 », etc.) au format ISO (« 1877-02-01 »)
    dt = dateparser.parse(date_str, languages=['fr'])
    if dt:
        return dt.strftime("%Y-%m-%d")
    return date_str

def update_triples_date_obj(triples):
    # Pour chaque triple, si l'objet est une date complète en français, on la remplace par le format ISO
    pattern = r"\b(?:1er|\d{1,2})\s+[a-zéû]+(?:\s+\d{4})"
    for triple in triples:
        obj = triple.get("obj", "")
        # Vérifie si l'objet est une date en français (« 1er février 1877 », « 21 mai 1956 », etc.)
        if re.fullmatch(pattern, obj, re.IGNORECASE):
            iso_date = fr_date_to_iso(obj)
            triple["obj"] = iso_date
    return triples

# ========== Traitement batch du fichier JSONL ==========
input_file = "C:/Users/jguo/Desktop/PURE-main/data/test_fixed.jsonl"
output_file = "C:/Users/jguo/Desktop/PURE-main/data/new_test.jsonl"

with open(input_file, "r", encoding="utf-8") as fin, open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        d = json.loads(line)
        d["triples"] = update_triples_date_obj(d["triples"])
        fout.write(json.dumps(d, ensure_ascii=False) + "\n")

print("Traitement terminé !")


Traitement terminé !


In [84]:
# prétraitement du jeu de donées pour supprimer des triplets raisonnés
#"sub": "Boulevard de Sébastopol", "rel": "hasGeometryChangeOn", "obj": "noTime"
# si le sujet et l'objet entre la relation "hasNewName" sont pareils, on supprime le triplet. {"sub": "Pont Alexandre III", "rel": "hasNewName", "obj": "pont Alexandre III"}
import json

# changer le chemin d'accès aux fichiers d'entrée et de sortie
input_file = "C:/Users/jguo/Desktop/PURE-main/data/new_test.jsonl"
output_file = "C:/Users/jguo/Desktop/PURE-main/data/test_filtered.jsonl"

with open(input_file, "r", encoding="utf-8") as fin, open(output_file, "w", encoding="utf-8") as fout:
    for line in fin:
        data = json.loads(line)
        if "triples" in data:
            new_triples = []
            for triple in data["triples"]:
                rel = triple.get("rel", "")
                obj = triple.get("obj", "")
                sub = triple.get("sub", "")

                # 1：sauter obj == noTime
                if obj == "noTime":
                    continue
                # 2：sauter hasNewName et sub.lower() == obj.lower()
                if rel == "hasNewName" and sub.strip().lower() == obj.strip().lower():
                    continue
                if rel == "hasOldName" and sub.strip().lower() == obj.strip().lower():
                    continue

                # garder ce triplet
                new_triples.append(triple)

            # actualiser les triplets
            data["triples"] = new_triples

        # Chaque ligne de sortie reste un dictionnaire JSON.
        fout.write(json.dumps(data, ensure_ascii=False) + "\n")

print(" Tous les filtres ont été appliqués avec succès. Le fichier de sortie a été enregistré à l’emplacement suivant ：", output_file)

 Tous les filtres ont été appliqués avec succès. Le fichier de sortie a été enregistré à l’emplacement suivant ： C:/Users/jguo/Desktop/PURE-main/data/test_filtered.jsonl


In [85]:
!pip install transformers

In [86]:
!pip install unidecode

In [87]:
!pip install dateparser


In [88]:
import json
from transformers import AutoTokenizer
from unidecode import unidecode
import dateparser
import difflib
import re

tokenizer = AutoTokenizer.from_pretrained("C:/Users/jguo/Desktop/PURE-main/camembert-base")

def normalize(text):
    return unidecode(text).lower().replace(" ", "").replace("’", "").replace("'", "")

def find_token_span(tokens, text, entity, entity_type=None):
    # Sauter les entités "thoroughfare" et "municipality" sans aucun affichage
    if entity_type in ("thoroughfare", "municipality"):
        return None, None
    # Appariement souple pour les codes de voies : "voie AH/15", "AH/15", etc.
    if entity_type == "landmark":
        code_pattern = r"[A-Z]{1,3}/\d{1,3}"
        entity_code = re.findall(code_pattern, entity)
        if entity_code:
            c = entity_code[0]
            idx = text.find(c)
            if idx != -1:
                return char_to_token_span(tokens, text, idx, idx + len(c))
        entity_words = entity.strip().split()
        if len(entity_words) >= 2:
            last_words = " ".join(entity_words[-2:])
            idx = text.lower().find(last_words.lower())
            if idx != -1:
                return char_to_token_span(tokens, text, idx, idx + len(last_words))
            last_word = entity_words[-1]
            idx2 = text.lower().find(last_word.lower())
            if idx2 != -1:
                left = max(0, idx2-6)
                match = re.search(r"(du|de|des|la|le)\s+" + re.escape(last_word), text[left:idx2+len(last_word)], re.IGNORECASE)
                if match:
                    span_start = left + match.start()
                    span_end = left + match.end()
                    return char_to_token_span(tokens, text, span_start, span_end)
    if entity_type == "date":
        century_match = re.fullmatch(r"C(\d{1,2})(e)?", entity, re.IGNORECASE)
        if century_match:
            century_num = int(century_match.group(1))
            roman_map = {
                1: "I", 2: "II", 3: "III", 4: "IV", 5: "V", 6: "VI", 7: "VII", 8: "VIII", 9: "IX", 10: "X",
                11: "XI", 12: "XII", 13: "XIII", 14: "XIV", 15: "XV", 16: "XVI", 17: "XVII", 18: "XVIII", 19: "XIX", 20: "XX"
            }
            roman = roman_map.get(century_num, "")
            candidates = []
            patterns = [
                rf"{century_num}e siècle",
                rf"{century_num} siècle",
                rf"{roman}e siècle",
                rf"{roman} siècle"
            ]
            for pat in patterns:
                idx = text.lower().find(pat.lower())
                if idx != -1:
                    candidates.append((idx, idx + len(pat)))
            if candidates:
                char_start, char_end = candidates[0]
                return char_to_token_span(tokens, text, char_start, char_end)
        mois_fr = ["janvier", "fevrier", "mars", "avril", "mai", "juin", "juillet",
                   "aout", "septembre", "octobre", "novembre", "decembre"]
        mois_fr_accent = ["janvier", "février", "mars", "avril", "mai", "juin", "juillet",
                          "août", "septembre", "octobre", "novembre", "décembre"]
        m_full = re.fullmatch(r"(\d{4})-(\d{2})-(\d{2})", entity)
        m_month = re.fullmatch(r"(\d{4})-(\d{2})", entity)
        m_year = re.fullmatch(r"(\d{4})", entity)
        year, month_idx, day = None, None, None
        if m_full:
            year = m_full.group(1)
            month_idx = int(m_full.group(2)) - 1
            day = int(m_full.group(3))
        elif m_month:
            year = m_month.group(1)
            month_idx = int(m_month.group(2)) - 1
        elif m_year:
            year = m_year.group(1)
        txt_norm = unidecode(text).lower()
        candidates = []
        if year and month_idx is not None:
            for mois in [mois_fr[month_idx], mois_fr_accent[month_idx]]:
                if day:
                    regexs = [
                        rf"\b{day}\s+{mois}\s+{year}\b",
                        rf"\b{str(day).zfill(2)}\s+{mois}\s+{year}\b",
                        rf"\b1er\s+{mois}\s+{year}\b" if day == 1 else "",
                    ]
                    for rgx in regexs:
                        if not rgx: continue
                        mobj = re.search(rgx, txt_norm)
                        if mobj:
                            candidates.append((mobj.start(), mobj.end()))
                rgx = rf"\b{mois}\s+{year}\b"
                mobj = re.search(rgx, txt_norm)
                if mobj:
                    candidates.append((mobj.start(), mobj.end()))
        elif year:
            mobj = re.search(rf"\b{year}\b", txt_norm)
            if mobj:
                candidates.append((mobj.start(), mobj.end()))
        if candidates:
            candidates.sort()
            char_start, char_end = candidates[0]
            return char_to_token_span(tokens, unidecode(text), char_start, char_end)
        if entity in text:
            char_start = text.index(entity)
            return char_to_token_span(tokens, text, char_start, char_start + len(entity))
        if entity_type is not None and entity_type not in ("thoroughfare", "municipality"):
            print(f"Entity [{entity}] not found in [{text}]")
        return None, None

    idx_raw = text.find(entity)
    if idx_raw != -1:
        char_start = idx_raw
        char_end = idx_raw + len(entity)
        return char_to_token_span(tokens, text, char_start, char_end)
    entity_norm = normalize(entity)
    text_norm = normalize(text)
    if entity_norm in text_norm:
        idx = text_norm.index(entity_norm)
        best_start, best_end = None, None
        for start in range(len(text)):
            for end in range(start + len(entity), min(len(text), start + len(entity) + 8)):
                frag = text[start:end]
                if normalize(frag) == entity_norm:
                    best_start, best_end = start, end
                    break
            if best_start is not None:
                break
        if best_start is not None and best_end is not None:
            return char_to_token_span(tokens, text, best_start, best_end)
    text_norm = normalize(text)
    entity_norm = normalize(entity)
    idx_norm = text_norm.find(entity_norm)
    if idx_norm != -1:
        best_start, best_end = None, None
        for start in range(len(text)):
            for end in range(start + 1, min(len(text), start + len(entity) + 8) + 1):
                frag = text[start:end]
                if normalize(frag) == entity_norm:
                    best_start, best_end = start, end
                    break
            if best_start is not None:
                break
        if best_start is not None and best_end is not None:
            return char_to_token_span(tokens, text, best_start, best_end)
        else:
            if entity_type is not None and entity_type not in ("thoroughfare", "municipality"):
                print(f"[AVERTISSEMENT] L'entité normalisée « {entity} » n'a pas été retrouvée dans le texte original « {text} ».")
    max_ratio = 0
    best_start, best_end = None, None
    for start in range(len(text)):
        for end in range(start + max(2, len(entity) - 4), min(len(text), start + len(entity) + 8)):
            frag = text[start:end]
            frag_norm = normalize(frag)
            if len(frag_norm) < 3:
                continue
            ratio = difflib.SequenceMatcher(None, frag_norm, entity_norm).ratio()
            if ratio > 0.82 and ratio > max_ratio:
                max_ratio = ratio
                best_start, best_end = start, end
    if best_start is not None and best_end is not None:
        if entity_type not in ("thoroughfare", "municipality"):
            print(f"[INFO] Appariement flou '{entity}' ≈ '{text[best_start:best_end]}', similarité={max_ratio:.2f}")
        return char_to_token_span(tokens, text, best_start, best_end)
    if entity_type is not None and entity_type not in ("thoroughfare", "municipality"):
           print(f"Entity [{entity}] not found in [{text}]")
    return None, None

def char_to_token_span(tokens, text, char_start, char_end):
    curr_char = 0
    start_token_idx = None
    end_token_idx = None
    for idx, token in enumerate(tokens):
        token_str = token.replace("▁", " ")
        token_str = token_str.strip()
        while curr_char < len(text) and text[curr_char].isspace():
            curr_char += 1
        token_begin = curr_char
        token_end = curr_char + len(token_str)
        if start_token_idx is None and token_begin <= char_start < token_end:
            start_token_idx = idx
        if token_begin < char_end <= token_end:
            end_token_idx = idx
        curr_char = token_end
    if start_token_idx is not None and end_token_idx is not None:
        return start_token_idx, end_token_idx
    return None, None

def find_sublist(lst, sub):
    """Trouve le début et la fin d'une sous-liste dans une liste."""
    for i in range(len(lst) - len(sub) + 1):
        if lst[i:i+len(sub)] == sub:
            return i, i + len(sub) - 1
    return -1, -1

def extract_time(sent):
    # Recherche d'expressions de dates simples dans la phrase
    import re
    m = re.search(r"\d{1,2} [a-zéû]+ \d{4}|\d{4}-\d{2}-\d{2}", sent)
    if m:
        return m.group(), m.start(), m.end()
    return None, -1, -1

def tag_event_type(sent):
    import re
    m = re.search(r"\|\|\s*([^\|]+?)\s*\|\|", sent)
    if m:
        event = m.group(1).strip()
        before_event = sent[:m.start()]
        return event, before_event, m.start(), m.end()
    return None, None, -1, -1

def extract_time(sent):
    m = re.search(r"\d{1,2} [a-zéû]+ \d{4}|\d{4}-\d{2}-\d{2}|\d{4}|C\d{1,2}(e)?", sent)
    if m:
        return m.group(), m.start(), m.end()
    return None, -1, -1

def build_ner(sent, tokens, triples=None):
    ner = []
    geo_prefixes = ["Rue", "Place", "Boulevard", "Quai", "Impasse", "Passage", "Allée", "Avenue", "Cours", "Chemin"]
    if triples:
        for triple in triples:
            if triple["rel"] == "isLandmarkTypeOf" or triple["rel"] in ("hasOldName", "hasNewName"):
                loc_text = triple["obj"]
                loc_tokens = tokenizer.tokenize(loc_text)
                start, end = find_sublist(tokens, loc_tokens)
                if start == -1:
                    continue
                found = False
                for prefix in geo_prefixes:
                    prefix_tokens = tokenizer.tokenize(prefix)
                    for i in range(len(loc_tokens) - len(prefix_tokens) + 1):
                        if [t.lower() for t in loc_tokens[i:i+len(prefix_tokens)]] == [t.lower() for t in prefix_tokens]:
                            ner.append([start+i, start+i+len(prefix_tokens)-1, "caractéristique géographique"])
                            if i+len(prefix_tokens) < len(loc_tokens):
                                ner.append([start+i+len(prefix_tokens), end, "nom"])
                            ner.append([start, end, "nom géographique"])
                            found = True
                            break
                    if found:
                        break
                if not found:
                    ner.append([start, end, "nom géographique"])
                    ner.append([end, end, "nom"])
    # 事件类型和时间的标注保持不变
    event, before_event, event_start, event_end = tag_event_type(sent)
    if event and event_start > 0:
        event_tokens = tokenizer.tokenize(event)
        start, end = find_sublist(tokens, event_tokens)
        if start != -1:
            ner.append([start, end, "EventType"])
    time_str, t_start, t_end = extract_time(sent)
    if time_str:
        sent_before_time = sent[:t_start]
        before_tokens = tokenizer.tokenize(sent_before_time, )
        time_tokens = tokenizer.tokenize(time_str, )
        ner.append([len(before_tokens), len(before_tokens)+len(time_tokens)-1, "Time"])
    return ner

def looks_like_date(s):
    """
    Détermine si la chaîne correspond à un format de date (français, ISO ou siècle).

    """
    s = s.strip()
    # ISO日期
    if re.fullmatch(r"\d{4}-\d{2}-\d{2}", s):
        return True
    # 法语日期：13 février 1911
    if re.fullmatch(r"\d{1,2} [a-zéû]+ \d{4}", s.lower()):
        return True
    # 法语仅年份
    if re.fullmatch(r"\d{4}", s):
        return True
    # 世纪表达 C13 或 C13e
    if re.fullmatch(r"C\d{1,2}(e)?", s, re.IGNORECASE):
        return True
    # 其他可扩展
    return False

def build_relations(sent, tokens, ner, triples):
    relations = []
    def get_span_from_ner(entity, label):
        # 优先返回 tokens拼接后和entity完全一致的区间
        best_span = None
        for span in ner:
            start, end, ner_label = span
            if ner_label == label:
                ner_text = "".join(tokens[start:end+1]).replace("▁", " ").strip().lower()
                entity_norm = entity.strip().lower().replace("’", "").replace("'", "")
                if entity_norm == ner_text:
                    return (start, end)
                # 记录最大区间
                if best_span is None or (end - start) > (best_span[1] - best_span[0]):
                    best_span = (start, end)
        # 如果没有完全匹配，返回最大区间
        return best_span

    for triple in triples:
        sub_span = None
        obj_span = None
        sub_type = triple.get("sub_type", "landmark")
        obj_type = "date" if looks_like_date(triple["obj"]) else triple.get("obj_type", "landmark")
        if sub_type in ("thoroughfare", "municipality") or obj_type in ("thoroughfare", "municipality"):
            continue

        if triple["rel"] == "isLandmarkTypeOf":
            sub_span = get_span_from_ner(triple["sub"], "caractéristique géographique")
            obj_span = get_span_from_ner(triple["obj"], "nom géographique")
            if sub_span and obj_span:
                relations.append([sub_span[0], sub_span[1], obj_span[0], obj_span[1], triple["rel"]])
            continue

                
        if triple["rel"] in ("hasGeometryChangeOn", "hasNameChangeOn", "isNumberedOn", "isClassifiedOn"):
            sub_span = get_span_from_ner(triple["sub"], "nom géographique")
            obj_span = get_span_from_ner(triple["obj"], "Time")
            if sub_span is None:
                sub_span = find_token_span(tokens, sent, triple["sub"], entity_type=sub_type)
            if obj_span is None:
                obj_span = find_token_span(tokens, sent, triple["obj"], entity_type=obj_type)
            if sub_span and sub_span[0] is not None and obj_span and obj_span[0] is not None:
                relations.append([sub_span[0], sub_span[1], obj_span[0], obj_span[1], triple["rel"]])
            continue

        # 其他关系（如 hasOldName、hasNewName、touches、within）保持原逻辑
        if triple["rel"] in ("hasOldName", "hasNewName", "touches", "within"):
            sub_span = get_span_from_ner(triple["sub"], "nom géographique")
            obj_span = get_span_from_ner(triple["obj"], "nom géographique")
            if sub_span is None:
                sub_span = find_token_span(tokens, sent, triple["sub"], entity_type=sub_type)
            if obj_span is None:
                obj_span = find_token_span(tokens, sent, triple["obj"], entity_type=obj_type)
            if sub_span and sub_span[0] is not None and obj_span and obj_span[0] is not None:
                relations.append([sub_span[0], sub_span[1], obj_span[0], obj_span[1], triple["rel"]])
            continue

        # 其他关系保持原逻辑
        sub_span = get_span_from_ner(triple["sub"], "nom géographique")
        obj_span = get_span_from_ner(triple["obj"], "Time")
        if sub_span is None:
            sub_span = find_token_span(tokens, sent, triple["sub"], entity_type=sub_type)
        if obj_span is None:
            obj_span = find_token_span(tokens, sent, triple["obj"], entity_type=obj_type)
        if sub_span and sub_span[0] is not None and obj_span and obj_span[0] is not None:
            relations.append([sub_span[0], sub_span[1], obj_span[0], obj_span[1], triple["rel"]])
    return relations


def build_ner(sent, tokens, triples=None):
    ner = []
    geo_prefixes = ["square", "Rue", "Place", "Boulevard", "Quai", "Impasse", "Passage", "Allée", "Avenue", "Cours", "Chemin"]
    if triples:
        for triple in triples:
            # 只要 obj 不是时间类型就标注
            if not looks_like_date(triple["obj"]):
                loc_text = triple["obj"]
                loc_tokens = tokenizer.tokenize(loc_text)
                start, end = find_sublist(tokens, loc_tokens)
                if start == -1:
                    continue
                found = False
                for prefix in geo_prefixes:
                    prefix_tokens = tokenizer.tokenize(prefix)
                    for i in range(len(loc_tokens) - len(prefix_tokens) + 1):
                        if [t.lower() for t in loc_tokens[i:i+len(prefix_tokens)]] == [t.lower() for t in prefix_tokens]:
                            ner.append([start+i, start+i+len(prefix_tokens)-1, "caractéristique géographique"])
                            if i+len(prefix_tokens) < len(loc_tokens):
                                ner.append([start+i+len(prefix_tokens), end, "nom"])
                            ner.append([start, end, "nom géographique"])
                            found = True
                            break
                    if found:
                        break
                if not found:
                    ner.append([start, end, "nom géographique"])
                    ner.append([end, end, "nom"])
    # 事件类型和时间的标注保持不变
    event, before_event, event_start, event_end = tag_event_type(sent)
    if event and event_start > 0:
        event_tokens = tokenizer.tokenize(event)
        start, end = find_sublist(tokens, event_tokens)
        if start != -1:
            ner.append([start, end, "EventType"])
    time_str, t_start, t_end = extract_time(sent)
    if time_str:
        sent_before_time = sent[:t_start]
        before_tokens = tokenizer.tokenize(sent_before_time, )
        time_tokens = tokenizer.tokenize(time_str, )
        ner.append([len(before_tokens), len(before_tokens)+len(time_tokens)-1, "Time"])
    return ner

# 修改 process_one_line 调用
def process_one_line(d):
    sent = d["sent"]
    doc_key = d["id"]
    tokens = tokenizer.tokenize(sent, )
    ner = build_ner(sent, tokens, d.get("triples", []))  # 传递 triples
    relations = build_relations(sent, tokens, ner, d.get("triples", []))
    return {
        "doc_key": doc_key,
        "dataset": "évolutions d'événements",
        "sentences": [tokens],
        "ner": [ner],
        "relations": [relations]
    }
input_file = "C:/Users/jguo/Desktop/PURE-main/data/test_filtered.jsonl"
output_file = "C:/Users/jguo/Desktop/PURE-main/data/test_pure.jsonl"

# Traitement batch des données : conversion et annotation
with open(input_file, "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]
with open(output_file, "w", encoding="utf-8") as fout:
    for data in dataset:
        ex = process_one_line(data)
        fout.write(json.dumps(ex, ensure_ascii=False) + "\n")